<a href="https://colab.research.google.com/github/HemanthS3149/zoho_intern_work/blob/Week-4/Pytorch_tutorial_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Working with Images & Logistic Regression in PyTorch**


Using PyTorch and logistic regression for image classification (MNIST Dataset)

Torchvision is a package in PyTorch ecosystem, designed primarily for CV tasks.It provides tools that facilitate manipulation,processing,handling of image data, which are commonly used in DL models for vision applications.

In [ ]:
import torch
import torchvision
from torchvision.datasets import MNIST

In [ ]:
#Downloading training dataset
dataset=MNIST(root='data/',download=True)

In [ ]:
len(dataset)

There are 60000 images to train the model with.

In [ ]:
test_dataset=MNIST(root='data/',train=False) #To retrive test data
len(test_dataset)

In [ ]:
dataset[0] #Image size is 28x28
#Class of image is PIL.Image.Image

In [ ]:
import matplotlib.pyplot as plt
image,label=dataset[0]
plt.imshow(image,cmap='gray')
print("Label:",label)

In [ ]:
image,label=dataset[90]
plt.imshow(image,cmap='gray')
print("Label:",label)

Pytorch does not know to work with images, but can work well with tensors. To solve this problem, we can specify a transform while creating our dataset

We shall use the ToTensor transform to convert images into PyTorch tensors

In [ ]:
import torchvision.transforms as transforms

In [ ]:
#MNIST dataset(images and labels)
dataset=MNIST(root='data/',train=True,transform=transforms.ToTensor())

In [ ]:
img_tensor,label=dataset[10]
print(img_tensor.shape,label)

This image is converted to a 1x28x28 tensor. 1st dimension tracks the color channels.

In [ ]:
print(img_tensor[0,10:15,10:15])#0th image tensor la, 10-15 rows and columns

In [ ]:
plt.imshow(img_tensor[0,10:15,10:15],cmap='gray')

In [ ]:
from torch.utils.data import random_split
train_ds,val_ds=random_split(dataset,[50000,10000]) #We take the val_ds from the training dataset
len(train_ds),len(val_ds)

In [ ]:
from torch.utils.data import DataLoader #Dataloader is used for batching and parallesim purposes
batch_size=128
train_loader=DataLoader(train_ds,batch_size,shuffle=True)#To ensure that batches generated in each epoch are different
val_loader=DataLoader(val_ds,batch_size)#Since val_ds is used only for evaluating the model, no need to shuffle images

We now use nn.Linear to create the LR model.Since nn.Linear expects the training example to be a vector, each 1x28x28 image tensor is flattened into vector of size 784(28x28)

The output for each image is a vector of size 10,with each element telling prob of 0 to 9

In [ ]:
import torch.nn as nn
input_size=28*28
num_classes=10
#Logistic regression model
model=nn.Linear(input_size,num_classes)

In [ ]:
print(model.weight.shape)
print(model.weight)

In [ ]:
print(model.bias.shape)
model.bias

There are 7850 parameters here. Now, try to generate some outputs for our model. Taking first batch of 100 images from dataset and passing into the model

In [ ]:
for images,labels in train_loader:
  print(labels)
  print(images.shape)
  outputs=model(images)
  print(outputs)
  break

In [ ]:
images.shape

In [ ]:
images.reshape(128,784).shape #from 128,1,28,28 we made it to 128,784 for the nn.layer to accept

Extending the nn.Module class from Pytorch to define a custom model


In [ ]:
class MnistModel(nn.Module):
  def __init__(self): #inside the __init__ method, we instantiate the weights and biases using nn.Linear()
    super().__init__() #calls the constructor of the base class to properly initialize the model
    self.linear=nn.Linear(input_size,num_classes)#defines a linear layer, which automatically initializes the weights and biases for the linear transformation

  def forward(self,xb):#We flatten the input tensor and pass it onto self.linear. It takes an input tensor xb and processes it through the model
    xb=xb.reshape(-1,784) #-1 refers to the batch_size
    out=self.linear(xb) #passes the flattened input through the linear layer,producing an output tensor
    return out
model=MnistModel()#defines a simple nn with a single linear layer, suitable for a basic Logistic regression classifier

In [ ]:
model.linear

In [ ]:
print(model.linear.weight.shape, model.linear.bias.shape)
list(model.parameters())

We can use our new custom model in the same way as before.

In [ ]:
for images,labels in train_loader:
  print(images.shape)
  outputs=model(images)
  break

print("Outputs.shape:",outputs.shape)
print("Sample outputs:\n",outputs[:3].data)

To convert the output rows into probabilities, we use the softmax function

In [ ]:
import torch.nn.functional as F

In [ ]:
outputs[0:2].data

In [ ]:
#Apply softmax for each output row
probs=F.softmax(outputs,dim=1)
print("Sample probablities:\n",probs[:2].data)
print("Sum: ",torch.sum(probs[0]).item())

In [ ]:
max_probs,preds=torch.max(probs,dim=1)
print(preds)
print(max_probs)

We will not get accurate results cause we randomly initialized weights and biases. We need to train the model, i.e adjust the weights using gradient descent to make better predictions

In [ ]:
labels

In [ ]:
outputs[0:2]

In [ ]:
torch.sum(preds==labels)

In [ ]:
def accuracy(outputs,labels):
  _,preds=torch.max(outputs,dim=1) #first tensor has maximum values along dim=1 and 2nd tensor has indices of the maximum values(which represent predicted classes)
  return torch.tensor(torch.sum(preds==labels).item()/len(preds))#_ indicates we are not interested in storing or using the first tensor(containing the max values)


In [ ]:
accuracy(outputs,labels)

We use cross-entropy as a loss function for classification problems

In [ ]:
loss_fn=F.cross_entropy

In [ ]:
#Loss for current batch of data
loss=loss_fn(outputs,labels)
print(loss)

Cross entropy is the -ve log of the predicted prob of the correct label averaged over all training examples

Model Training


for epoch in range(num_epochs):
 #Training phase
* for batch in train_loader
*   Generate preds

*   Calculate loss
*   Compute gradients


*   Update weights

*   Reset Gradients

  #Validation Phase


for batch in val_loader:
*   Generate preds
*   Calculate loss
*   Calculate metrics(accuracy...)










In [ ]:
class MnistModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear=nn.Linear(input_size,num_classes)#Single layer for imp logistic regression

  def forward(self,xb):
    xb=xb.reshape(-1,784)
    out=self.linear(xb)
    return out

  def training_step(self,batch):
    images,labels=batch
    out=self(images)
    loss=F.cross_entropy(out,labels)
    return loss

  def validation_step(self,batch):
    images,labels=batch
    out=self(images)
    loss=F.cross_entropy(out,labels)
    acc=accuracy(out,labels)
    return {'val_loss':loss,'val_acc':acc}

  def validation_epoch_end(self,outputs):
    batch_losses=[x['val_loss'] for x in outputs]
    epoch_loss=torch.stack(batch_losses).mean() #mean of all losses
    batch_accs=[x['val_acc'] for x in outputs]
    epoch_acc=torch.stack(batch_accs).mean() #mean of all accuracies
    return {'val_loss':epoch_loss.item(),'val_acc':epoch_acc.item()}

  def epoch_end(self, epoch, result):
        print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result['val_loss'], result['val_acc']))

model=MnistModel()


Now defining an evaluate function, which performs validation phase and a fit func which will perform the entire training process

In [ ]:
def evaluate(model,val_loader):
  outputs=[model.validation_step(batch) for batch in val_loader]
  return model.validation_epoch_end(outputs)

def fit(epochs,lr,model,train_loader,val_loader,opt_fun=torch.optim.SGD):
  history=[]
  optimizer=opt_fun(model.parameters(),lr) #we pass on w and b and learning rate to the opt_fun
  for epoch in range(epochs):
    #Training phase
    for batch in train_loader:
      loss=model.training_step(batch)
      loss.backward() #compulsory in all cases
      optimizer.step()
      optimizer.zero_grad() #resets the gradients to 0
     #Validation phase
    result=evaluate(model,val_loader)
    model.epoch_end(epoch,result)
    history.append(result)
  return history


In [ ]:
result0=evaluate(model,val_loader)
result0

In [ ]:
history1=fit(5,0.001,model,train_loader,val_loader)

Epoch[0],val_loss:2.3191,val_acc:0.1301
Epoch[0],val_loss:2.3180,val_acc:0.1307
Epoch[0],val_loss:2.3167,val_acc:0.1319
Epoch[0],val_loss:2.3157,val_acc:0.1330
Epoch[0],val_loss:2.3147,val_acc:0.1342
Epoch[0],val_loss:2.3135,val_acc:0.1340
Epoch[0],val_loss:2.3125,val_acc:0.1357
Epoch[0],val_loss:2.3114,val_acc:0.1378
Epoch[0],val_loss:2.3101,val_acc:0.1397
Epoch[0],val_loss:2.3089,val_acc:0.1405
Epoch[0],val_loss:2.3079,val_acc:0.1417
Epoch[0],val_loss:2.3068,val_acc:0.1424
Epoch[0],val_loss:2.3058,val_acc:0.1433
Epoch[0],val_loss:2.3046,val_acc:0.1446
Epoch[0],val_loss:2.3035,val_acc:0.1455
Epoch[0],val_loss:2.3025,val_acc:0.1469
Epoch[0],val_loss:2.3015,val_acc:0.1479
Epoch[0],val_loss:2.3004,val_acc:0.1495
Epoch[0],val_loss:2.2993,val_acc:0.1499
Epoch[0],val_loss:2.2981,val_acc:0.1510
Epoch[0],val_loss:2.2968,val_acc:0.1513
Epoch[0],val_loss:2.2955,val_acc:0.1525
Epoch[0],val_loss:2.2943,val_acc:0.1535
Epoch[0],val_loss:2.2932,val_acc:0.1546
Epoch[0],val_loss:2.2922,val_acc:0.1558


In [ ]:
history=[result0]#+history1+history2+history3+history4
accuracies=[result['val_acc'] for result in history]
plt.plot(accuracy,'-x')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.title('Accuracy vs No. of epochs')

**Testing with individual images**

In [ ]:
#Defining test dataset
test_dataset=MNIST(root='data/',train=False,transform=transforms.ToTensor())

In [ ]:
img,label=test_dataset[0]
plt.imshow(img[0],cmap='gray')
print("Shape:",img.shape)
print("Label:",label)

In [ ]:
def predict_image(img,model):#Returns predicted image label for a single image tensor
  xb=img.unsqueeze(0)#adds another dimension at the start of 1x28x28 to make it 1x1x28x28 which the model views as a batch containing a single image
  yb=model(xb)
  _,preds=torch.max(yb,dim=1)
  return preds[0].item()

In [ ]:
img,label=test_dataset[0]
plt.imshow(img[0],cmap='gray')
print("Label:",label,',Predicted:',predict_image(img,model))

In [ ]:
img,label=test_dataset[10]
plt.imshow(img[0],cmap='gray')
print('Label:',label,',Predicted:',predict_image(img,model))

Viewing the overall loss and accuracy of the model on test set


In [ ]:
test_loader=DataLoader(test_dataset,batch_size=256)
result=evaluate(model,test_loader)
result